In [41]:
import matplotlib.pyplot as plt
import numpy as np
import collections
import random
from tqdm.auto import tqdm


%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

Let's take a look at the data:

In [42]:
with open('text_seg/data_small.txt', 'r', encoding='utf-8') as file:
    text = file.read()
    print(text[:1000])

TributespouredinfromaroundtheworldThursdaytothelateLabourPartyleaderJohnSmith,whodiedearlierfromamassiveheartattackaged55.InWashington,theUSStateDepartmentissuedastatementregretting"theuntimelydeath"oftherapier-tonguedScottishbarristerandparliamentarian."Mr.Smith,throughouthisdistinguishedcareeringovernmentandinopposition,leftaprofoundimpressiononthehistoryofhispartyandhiscountry,"StateDepartmentspokesmanMichaelMcCurrysaid."Secretary(ofStateWarren)ChristopherextendshisdeepestcondolencestoMrs.SmithandtotheSmithchildren."InBonn,theheadoftheGermanSocialDemocraticParty,RudolfScharping,saidinastatementhewas"veryaffectedbythesuddendeathofJohnSmith."AgoodfriendofGermansocialdemocracyhasleftustooearly.Hewasveryclosetoachievinghislife'sgoalofmakingtheLabourPartythelargestpoliticalforceinBritain"andwouldbe"cruellymissed"inEurope,hesaid.HongKongGovernorChrisPatten,aformerConservativePartychairman,offeredhiscondolencestotheSmithfamilyandsaidhisformerpolitcalopponentwasa"goodanddecentman,widelyresp

In [43]:
text_size = len(text)
text_size

652297

In [44]:
C = len(set(text))

In [45]:
print("Number of unique characters: {}".format(C))

Number of unique characters: 74


- There is 2 𝑛−1 possible segmentations for 𝑛-characters long data.
- 𝑛 − 1 latent binary variables $𝑠_𝑖$: denoting whether there is of isn’t a separator between two characters.
- Collapsed Gibbs sampling. Sample one variable conditioned by all the others.
- Exchangeability: if we reorder the words in the sequence, overall probability is the same.
- We can virtually move the changed words at the end of the sequence, compute the overal probablility of the two possibilities and then move the words virtually back.


if $s_i$ is 1, then there is a separator between characters $c_i$ and $c_{i+1}$

In [46]:
# Fixing the random seeds
np.random.seed(1234)
random.seed(1234)

In [47]:
import collections

def segmentation(text, text_size, s):
    words = []
    current_word = ""
    for idx, character in enumerate(text):
        if idx == text_size - 1:
            current_word += character
            continue
        
        if s[idx] == 1:
            current_word += character
            words.append(current_word)
            current_word = ""
        else:
            current_word += character
    return words

def get_word_counts(words):
    count = {}
    counter = collections.Counter(words)
    return counter
    for key, value in counter.items():
        count[key] = value
    return count

In [48]:
def get_prev_word(text, s, i):
    if i == 0:
        return text[i]
    
    start_idx = i
    end_idx = i + 1
    
    # Edge cases
    if s[start_idx] == 1 and s[start_idx-1] == 1:
        return text[i]
    
    if s[start_idx] == 1 and s[start_idx-1] == 0:
        start_idx -= 1
        
    while start_idx >= 0:
        if s[start_idx] == 1:
            start_idx += 1
            break
        start_idx -= 1
    
    if start_idx < 0:
        start_idx = 0
        
    return text[start_idx:end_idx]

In [49]:
def get_next_word(text, s, i):
    word = ""
    start_idx = i + 1
    end_idx = i + 1
    while end_idx <= len(text) - 1:
        if s[end_idx] == 1:
            break
        end_idx += 1
    
    if end_idx == len(text):
        end_idx -= 1
    
    word = text[start_idx:end_idx+1]
    return word

In [50]:
for i in range(7):
    prev = get_prev_word("Tributes", [0, 0, 1, 1, 0, 1, 0, 1], i)
    nexts = get_next_word("Tributes", [0, 0, 1, 1, 0, 1, 0, 1], i)
    print("{}: {}\t{}".format(i, prev, nexts))

0: T	ri
1: Tr	i
2: Tri	b
3: b	ut
4: u	t
5: ut	es
6: e	s


In [51]:
def p0(word, p_c):
    uniform = 1.0 / float(C)
    return uniform**len(word) * p_c**(len(word)-1) * (1 - p_c)

In [52]:
def CRPTextSegmentation(text, text_size, iterations, alpha, p_c, p_cont, T = 1, T_decrease = 1):
    # randomly initialize text segmentation
    s = np.random.randint(low=0, high=2, size=text_size)
    
    # create the initial segmentation
    words = segmentation(text, text_size, s)
    count = get_word_counts(words)
    
    # total number of words
    t = sum(count.values())
    
    processing_progress_bar = tqdm(range(1, text_size - 1), desc="Text processing")
    
    for iteration in tqdm(range(iterations), desc="Iterations"):
        processing_progress_bar.reset()
        for i in np.random.permutation(range(0, text_size - 1)):
            
            prev_word = get_prev_word(text, s, i)
            next_word = get_next_word(text, s, i)
            
            joined = prev_word + next_word
            if s[i] == 0:
                if count[joined] == 0:
                    print(joined + " not found")
                    print(count)
                count[joined] = max(0, count[joined] - 1)
                t -= 1
            else:
                count[prev_word] = max(0, count[prev_word] - 1)
                count[next_word] = max(0, count[next_word] - 1)
                t -= 2
            
            p_0 = (alpha * p0(joined, p_c) + count[joined]) / (alpha + t)
            p_1 = (alpha * p0(prev_word, p_c) + count[prev_word]) / (alpha + t)
            p_1 *= (alpha * p0(next_word, p_c) + count[next_word]) / (alpha + t + 1)
            p_1 *= p_cont
            
            # Annealing
            p_0 = p_0 ** (1/T)
            p_1 = p_1 ** (1/T)
            
            # Normalization
            suma = p_0 + p_1
            p_0 /= suma
            p_1 /= suma
            
            s[i] = np.random.choice([0, 1], p=[p_0, p_1])
            
            if s[i] == 0:
                count[joined] += 1
                t += 1
            else:
                count[prev_word] += 1
                count[next_word] += 1
                t += 2

            processing_progress_bar.update(1)
        
    words_updated = segmentation(text, text_size, s)
    final_output = " ".join(words_updated)
    return final_output

## Task 3

**Download the gold data and the evaluation script. What precision and recall you get?**

In [53]:
def save_and_evaluate(output, output_file_path):
    output_file_path = "text_seg/" + output_file_path
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(output)
    !perl text_seg/eval.pl text_seg/data_small_gold.txt $output_file_path
    
    with open('text_seg/results.txt', 'r') as f_results:
        lines = f_results.readlines()
        precision = round(float(lines[0]), 4)
        recall = round(float(lines[1]), 4)
        f1 = round(float(lines[2]), 4)
        
        return precision, recall, f1

In [54]:
def fit(run_name, iterations, alpha, p_c, p_cont, T=1, T_decrease=1):
    output = CRPTextSegmentation(text, text_size, iterations, alpha, p_c, p_cont, T, T_decrease)
    
    output_file_path = "{}.txt".format(run_name)
    print("Evaluating run: {} in file {}".format(run_name, output_file_path))
    precision, recall, f1 = save_and_evaluate(output, output_file_path)
    
    return precision, recall, f1

## Trying out different parameters

- ... try to change the parameters to obtain better segmentations
- What precision and recall you get?
- Try to do annealing and run the model for different temperatures.

In [ ]:
import csv

runs = []
best_run = None

with open('text_seg/config.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        name = row["name"]
        iterations = int(row["iterations"])
        alpha = int(row["alpha"])
        p_c = float(row["p_c"])
        p_cont = float(row["p_cont"])
        T = float(row["T"])
        T_decrease = float(row["T_decrease"])
        
        print("Currently working on: {}".format(name))
        print("Configuration:")
        print(iterations, alpha, p_c, p_cont, T, T_decrease)
        precision, recall, f1 = fit(
            name, 
            iterations, 
            alpha, 
            p_c, 
            p_cont, 
            T, 
            T_decrease)
        
        config_and_results = (name, iterations, alpha, p_c, p_cont,
                              T, T_decrease, precision, recall, f1)
        
        runs.append(config_and_results)
        
        if best_run == None:
            best_run = config_and_results
        else:
            if best_run[9] < config_and_results[9]:
                best_run = config_and_results
        

Currently working on: basic
Configuration:
100 100 0.5 0.99 1.0 1.0


Text processing:   0%|          | 0/652295 [00:00<?, ?it/s]

Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluating run: basic in file basic.txt
P:0.172, R:0.289, F:0.216
Currently working on: run2
Configuration:
100 100 0.4 0.99 1.0 1.0


Text processing:   0%|          | 0/652295 [00:00<?, ?it/s]

Iterations:   0%|          | 0/100 [00:00<?, ?it/s]

es.
 not found
Counter({'e': 18051, 'a': 12803, 't': 12530, 'i': 11498, 'n': 10835, 'o': 10675, 'r': 9862, 's': 9421, 'h': 6362, 'd': 6091, 'l': 6019, 'c': 4243, 'u': 3541, 'm': 3298, 'f': 3169, 'p': 2848, 'g': 2655, 'y': 2298, 'w': 2273, 'b': 1795, 'th': 1760, 'in': 1692, 'he': 1683, ',': 1576, '.': 1454, 'er': 1446, 'an': 1273, 'v': 1252, 're': 1138, 'on': 1135, 'es': 991, 'k': 962, 'nt': 942, 'ed': 935, 'en': 914, 'st': 859, 'to': 826, 'at': 795, 'nd': 794, 'ar': 788, 'ti': 778, 'te': 775, 'or': 774, 'the': 764, 'al': 737, '"': 653, 'ng': 637, 'it': 620, 'ea': 619, 'is': 595, 'ou': 584, 'ri': 583, 'of': 577, 'A': 573, 'S': 566, 'T': 565, 'as': 561, 'sa': 553, 'ra': 540, 'ha': 536, 'de': 532, 'me': 531, 'le': 508, 'ta': 501, 'se': 498, 've': 492, 'li': 489, '-': 488, 'ro': 468, 'ce': 467, 'io': 461, 'ne': 460, 'si': 456, '0': 453, 'ic': 441, 'et': 440, 'la': 419, 'ec': 409, 'na': 408, 'id': 405, 'da': 402, 'co': 400, 'ai': 387, 'll': 387, 'ur': 386, 'di': 380, '1': 375, 'ni': 373, 'I

In [ ]:
import pandas as pd

df = pd.DataFrame.from_records(runs, columns =['Run name', 'Iterations', 'alpha', 'p_c', 'p_cont', 
                                              'T', 'T_decrease', 'Precision', 'Recall', 'F1'])

In [ ]:
df

## Best run:

In [ ]:
print(best_run)

## Task 5

Instead of Chinese Restaurant Process, try to employ the Pitman-Yor Process. Does it improve your results?

In [ ]:
def PitmanYorTextSegmentation(text, text_size, iterations, alpha, p_c, p_cont, T = 1, T_decrease = 1):
    # randomly initialize text segmentation
    s = np.random.randint(low=0, high=2, size=text_size)
    
    # create the initial segmentation
    words = segmentation(text, text_size, s)
    count = get_word_counts(words)
    
    # total number of words
    t = sum(count.values())
    
    processing_progress_bar = tqdm(range(1, text_size - 1), desc="Text processing")
    
    for iteration in tqdm(range(iterations), desc="Iterations"):
        processing_progress_bar.reset()
        for i in np.random.permutation(range(0, text_size - 1)):
            
            prev_word = get_prev_word(text, s, i)
            next_word = get_next_word(text, s, i)
            
            joined = prev_word + next_word
            if s[i] == 0:
                if count[joined] == 0:
                    print(joined + " not found")
                    print(count)
                count[joined] = max(0, count[joined] - 1)
                t -= 1
            else:
                count[prev_word] = max(0, count[prev_word] - 1)
                count[next_word] = max(0, count[next_word] - 1)
                t -= 2
            
            p_0 = (alpha * p0(joined, p_c) + count[joined]) / (alpha + t)
            p_1 = (alpha * p0(prev_word, p_c) + count[prev_word]) / (alpha + t)
            p_1 *= (alpha * p0(next_word, p_c) + count[next_word]) / (alpha + t + 1)
            p_1 *= p_cont
            
            # Annealing
            p_0 = p_0 ** (1/T)
            p_1 = p_1 ** (1/T)
            
            # Normalization
            suma = p_0 + p_1
            p_0 /= suma
            p_1 /= suma
            
            s[i] = np.random.choice([0, 1], p=[p_0, p_1])
            
            if s[i] == 0:
                count[joined] += 1
                t += 1
            else:
                count[prev_word] += 1
                count[next_word] += 1
                t += 2

            processing_progress_bar.update(1)
        
    words_updated = segmentation(text, text_size, s)
    final_output = " ".join(words_updated)
    return final_output